In [1]:
from typing import TypedDict,List,Annotated
from langgraph.graph import END,StateGraph
import operator

In [2]:
from typing import TypedDict

class User(TypedDict):
    name: str
    age: int

person: User = {
    "name": "Dulana",
    "age": 22
}


In [3]:
from typing import List

def sum_all(values: List[int]) -> int:
    return sum(values)

print(sum_all([1, 2, 3]))  # 6


6


3) Annotated

Allows you to attach metadata or constraints to a type.

Think of it as:

A type + extra info about that type

LangGraph uses Annotated to mark fields that should be tracked, merged, or updated across nodes.

In [4]:
from typing import Annotated

Age = Annotated[int, "must be positive"]


In [6]:
from typing import Annotated
from langgraph.graph import add_messages

class ChatState(TypedDict):
    messages: Annotated[List[str], add_messages]

"""Meaning:
    messages is a list of strings
    LangGraph should append NEW messages to the existing list automatically
"""

'Meaning:\n    messages is a list of strings\n    LangGraph should append NEW messages to the existing list automatically\n'

In [7]:
import operator

print(operator.add(3, 4))  # 7


7


In [8]:
from operator import itemgetter

items = [{"x": 5}, {"x": 3}, {"x": 10}]
sorted_items = sorted(items, key=itemgetter("x"))

print(sorted_items)


[{'x': 3}, {'x': 5}, {'x': 10}]


In [9]:
from typing import TypedDict,List,Annotated
from langgraph.graph import END,StateGraph
import operator

class SimpleState(TypedDict):
    count: int
    sum: Annotated[int,operator.add]
    history: Annotated[List[int],operator.concat]

def increment(state: SimpleState) -> SimpleState:
    new_count = state['count'] + 1

    return {
        "count": new_count,
        "sum": new_count,
        "history": [new_count]
    }

def should_continue(state):
    if(state["count"] < 5):
        return "continue"
    else:
        return "stop"

In [14]:
graph = StateGraph(SimpleState)
graph.add_node("increment",increment)
graph.set_entry_point("increment")
graph.add_conditional_edges(
    "increment",
    should_continue,
    {
        "continue": "increment",
        "stop": END
    }
)
app = graph.compile()

state = {
    "count": 0,
    "sum": 0,
    "history": []
}

result = app.invoke(state)
print(result)

{'count': 5, 'sum': 15, 'history': [1, 2, 3, 4, 5]}
